<a href="https://colab.research.google.com/github/AbeHandler/AbeHandler.github.io/blob/master/Katie_AMJ_intuition_building_implementation_of_Egami.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from random import random


def llm(row, femaleaccuracy, maleaccuracy):
    if row["female"] == 1:
        if random() < femaleaccuracy:
            return row["DV"]
        else:
            return 1 - row["DV"]
    else:
        if random() < maleaccuracy:
            return row["DV"]
        else:
            return 1 - row["DV"]

def generate_data(N, pct_female,
                  beta1=0.5,
                  beta2=-.5,
                  intercept=0.0,
                  errorsd=0.1,
                  femaleaccuracy=1,
                  maleaccuracy=1,
                  seed=None):
    np.random.seed(seed)

    public_salary = np.random.binomial(1, 0.5, size=N)

    female = np.random.binomial(1, pct_female, size=N)

    error = np.random.normal(0, errorsd, size=N)

    linear_predictor = intercept + public_salary * beta1 + female * beta2 + error

    # for logistic regression
    prob = 1 / (1 + np.exp(-linear_predictor))
    DV = np.random.binomial(1, prob)

    # for linear regression
    DV = (linear_predictor > 0).astype(int)

    data = pd.DataFrame({
        'public_salary': public_salary,
        'female': female,
        'DV': DV
    })

    data["predicted_DV"] = data.apply(lambda row: llm(row, femaleaccuracy=femaleaccuracy, maleaccuracy=maleaccuracy), axis=1)
    return data


L = []
I = []
for i in range(1000):
    N = 1000
    Nannotated = 100
    pi = Nannotated/N
    pct_female = 0.25
    generated_data = generate_data(N, pct_female, femaleaccuracy=.75, maleaccuracy=.9)
    generated_data["expert"] = np.random.binomial(1, pi, size=N)
    log_reg = LinearRegression()
    X = generated_data[['public_salary', 'female']]
    y = generated_data['predicted_DV']
    yadjusted = y - ((generated_data["expert"]/pi) * (generated_data["predicted_DV"] - generated_data["DV"]))

    log_reg.fit(X, yadjusted)
    coefficients = log_reg.coef_
    intercept = log_reg.intercept_
    I.append(intercept)
    L.append(coefficients)


matrix = np.vstack(L)
matrix[:,1].mean()
np.mean(I)

0.5015824597644563

In [125]:
matrix[:,0].mean()

0.4986333086156519

### How does this help theory

- Better measurements. What is related to what
- Text added to DAG. New DVs.